In [216]:
import sympy as sp
import itertools
import pypolsys
import numpy as np
import numpy as np
from itertools import chain, combinations, product
from typing import List, Iterator, Tuple, Any, Generator

In [12]:
class Game:
    def __init__(self, num_players, num_strategies, payoffs):
        self.num_players = num_players
        self.num_strategies = num_strategies
        self.payoffs = payoffs
        #self.indices_combinations = indices_combinations


def create_n_player_game(payoffs):
    num_players = len(payoffs)
    num_strategies = [len(payoffs[i]) for i in range(num_players)]
    #indices_combinations = list(itertools.product(*[range(num_players)] * len(num_strategies)))

    return Game(num_players, num_strategies, payoffs)

In [252]:
payoffs = [
    [
        [
            [9, 0],
            [0, 3]
        ],
        [
            [0, 3],
            [9, 0]
        ]
    ],
    [
        [
            [8, 0],
            [0, 4]
        ],
        [
            [0, 4],
            [8, 0]
        ]
    ],
    [
        [
            [12, 0],
            [0, 2]
        ],
        [
            [0, 6],
            [6, 0]
        ]
    ]
    ]

game = create_n_player_game(payoffs)
game.num_players, game.num_strategies, game.payoffs[0][0][0]

(3, [2, 2, 2], [9, 0])

list

In [251]:
payoffs2 = [
    [
        [
            1,
            0,
            0
        ],
        [
            0,
            1,
            0
        ],
        [
            0,
            0,
            1
        ]
    ],
    [
        [
            1,
            0,
            0
        ],
        [
            0,
            1,
            0
        ],
        [
            0,
            0,
            1
        ]
    ]
]

game2 = create_n_player_game(payoffs2)
game2.num_players, game2.num_strategies, game2.payoffs

(2,
 [3, 3],
 [[[1, 0, 0], [0, 1, 0], [0, 0, 1]], [[1, 0, 0], [0, 1, 0], [0, 0, 1]]])

In [29]:
payoffs3 = [
    [
        [
            [9, 0],
            [0, 3]
        ],
        [
            [0, 3],
            [9, 0]
        ]
        ,
        [
            [0, 1],
            [2, 3]
        ]
    ],
    [
        [
            [8, 0],
            [0, 4],
            [2, 3]
        ],
        [
            [0, 4],
            [8, 0],
            [5, 7]
        ]
    ],
    [
        [
            [12, 0],
            [0, 2],
            [1, 1]
        ],
        [
            [0, 6],
            [6, 0],
            [2, 3]
        ]
    ]
]

game3 = create_n_player_game(payoffs3)
game3.num_players, game3.num_strategies, game3.payoffs[0][0][0][0]

(3, [3, 2, 2], 9)

In [30]:
payoffs4 = [
    np.array([
        [[[1, 2], [3, 4]], [[5, 6], [7, 8]]],
        [[[9, 10], [11, 12]], [[13, 14], [15, 16]]]
    ]),
    np.array([
        [[[17, 18], [19, 20]], [[21, 22], [23, 24]]],
        [[[25, 26], [27, 28]], [[29, 30], [31, 32]]]
    ]),
    np.array([
        [[[33, 34], [35, 36]], [[37, 38], [39, 40]]],
        [[[41, 42], [43, 44]], [[45, 46], [47, 48]]]
    ]),
    np.array([
        [[[49, 50], [51, 52]], [[53, 54], [55, 56]]],
        [[[57, 58], [59, 60]], [[61, 62], [63, 64]]]
    ])
]
game4 = create_n_player_game(payoffs4)
game4.num_players, game4.num_strategies, game4.payoffs

(4,
 [2, 2, 2, 2],
 [array([[[[ 1,  2],
           [ 3,  4]],
  
          [[ 5,  6],
           [ 7,  8]]],
  
  
         [[[ 9, 10],
           [11, 12]],
  
          [[13, 14],
           [15, 16]]]]),
  array([[[[17, 18],
           [19, 20]],
  
          [[21, 22],
           [23, 24]]],
  
  
         [[[25, 26],
           [27, 28]],
  
          [[29, 30],
           [31, 32]]]]),
  array([[[[33, 34],
           [35, 36]],
  
          [[37, 38],
           [39, 40]]],
  
  
         [[[41, 42],
           [43, 44]],
  
          [[45, 46],
           [47, 48]]]]),
  array([[[[49, 50],
           [51, 52]],
  
          [[53, 54],
           [55, 56]]],
  
  
         [[[57, 58],
           [59, 60]],
  
          [[61, 62],
           [63, 64]]]])])

In [25]:
def calculate_expected_payoff_diff_equations(game):
    n = game.num_players
    # Create symbolic probability variables
    probabilities = [[sp.Symbol(f'p{i+1}{j+1}') for j in range(game.num_strategies[i])] for i in range(n)]

    def expected_payoff(player, strategy):
        s = game.num_strategies[player]
        payoff = 0

        # Create a modified list of strategy ranges, replacing the player's strategy range with [strategy]
        strategy_ranges = [range(game.num_strategies[p]) if p != player else [strategy] for p in range(n)]
        for indices in itertools.product(*strategy_ranges):
            probability_combination = [probabilities[p][indices[p]] if p != player else 1 for p in range(n)]
            payoff_value = game.payoffs[player][strategy]
            for i, index in enumerate(indices):
                if i != player:
                    payoff_value = payoff_value[index]
            payoff_term = payoff_value * sp.prod(probability_combination)

            payoff += payoff_term
        return payoff

    payoff_differences_equations = []
    for player in range(n):
        player_payoff_diff = []
        for strategy in range(game.num_strategies[player]-1):
            diff = expected_payoff(player, strategy) - expected_payoff(player, game.num_strategies[player]-1)
            player_payoff_diff.append(diff)
        payoff_differences_equations.append(player_payoff_diff)

    # Flatten the list of lists
    flat_payoff_differences_equations = [diff for sublist in payoff_differences_equations for diff in sublist]

    return flat_payoff_differences_equations

In [161]:
calculate_payoff_diff_and_prob_sum_equations(game2)

[Poly(p21 - p23, p11, p12, p13, p21, p22, p23, domain='ZZ'),
 Poly(p22 - p23, p11, p12, p13, p21, p22, p23, domain='ZZ'),
 Poly(p11 - p13, p11, p12, p13, p21, p22, p23, domain='ZZ'),
 Poly(p12 - p13, p11, p12, p13, p21, p22, p23, domain='ZZ'),
 Poly(p11 + p12 + p13 - 1, p11, p12, p13, p21, p22, p23, domain='ZZ'),
 Poly(p21 + p22 + p23 - 1, p11, p12, p13, p21, p22, p23, domain='ZZ')]

In [20]:
def calculate_expected_payoff_diff_equations_fj(game):
    n = game.num_players
    # Create symbolic probability variables
    probabilities = [[sp.Symbol(f'p{i+1}{j+1}') for j in range(game.num_strategies[i])] for i in range(n)]

    def expected_payoff(player, strategy):
        s = game.num_strategies[player]
        payoff = 0

        # Create a modified list of strategy ranges, replacing the player's strategy range with [strategy]
        strategy_ranges = [range(game.num_strategies[p]) if p != player else [strategy] for p in range(n)]
        for indices in itertools.product(*strategy_ranges):
            probability_combination = [probabilities[p][indices[p]] if p != player else 1 for p in range(n)]
            payoff_value = game.payoffs[player][strategy]
            for i, index in enumerate(indices):
                if i != player:
                    payoff_value = payoff_value[index]
            payoff_term = payoff_value * sp.prod(probability_combination)

            payoff += payoff_term
        return payoff

    def expected_payoff_from_mixed_strategy(player):
        s = game.num_strategies[player]
        total_payoff = 0
        for strategy in range(s):
            total_payoff += probabilities[player][strategy] * expected_payoff(player, strategy)
        return total_payoff

    payoff_differences_equations = []
    for player in range(n):
        player_payoff_diff = []
        for strategy in range(game.num_strategies[player]):
            diff = expected_payoff(player, strategy) - expected_payoff_from_mixed_strategy(player)
            player_payoff_diff.append(diff)
        payoff_differences_equations.append(player_payoff_diff)

    # Flatten the list of lists
    flat_payoff_differences_equations = [diff for sublist in payoff_differences_equations for diff in sublist]

    return flat_payoff_differences_equations

In [21]:
def calculate_jacobian(game):
    # Get the flattened payoff difference equations
    payoff_differences_equations = calculate_expected_payoff_diff_equations_fj(game)
    
    n = game.num_players
    probabilities = [[sp.Symbol(f'p{i+1}{j+1}') for j in range(game.num_strategies[i])] for i in range(n)]

    # Flatten the list of probability variables
    flat_probabilities = [p for sublist in probabilities for p in sublist]

    # Calculate the Jacobian
    jacobian_matrix = sp.Matrix(payoff_differences_equations).jacobian(flat_probabilities)

    return jacobian_matrix

In [22]:
def calculate_payoff_diff_and_prob_sum_equations(game):
    n = game.num_players
    probabilities = [[sp.Symbol(f'p{i+1}{j+1}') for j in range(game.num_strategies[i])] for i in range(n)]

    # Collect the equations in a list
    equations = []
    for player_payoff_diff in calculate_expected_payoff_diff_equations(game):
        equations.append(sp.poly(player_payoff_diff , *[p for player_probs in probabilities for p in player_probs]))

    for player_probs in probabilities:
        prob_sum_eq = sum(player_probs) - 1
        equations.append(sp.poly(prob_sum_eq, *[p for player_probs in probabilities for p in player_probs]))


    return equations

In [23]:
def nash_solver(game, tol):
    D = sum(game.num_strategies)
    pol = pypolsys.utils.fromSympy(calculate_payoff_diff_and_prob_sum_equations(game))
    pypolsys.polsys.init_poly(*pol)
    part = pypolsys.utils.make_h_part(D)
    pypolsys.polsys.init_partition(*part)
    bplp = pypolsys.polsys.solve(1e-8, 1e-15, 0.0)
    r = pypolsys.polsys.myroots
    def remove_near_zero_complex(array, tol):
        # Check if the imaginary or real part of any element in a row is close to zero
        is_complex_close_to_zero = lambda row: any(x.real <= 0 or abs(x.imag) > tol for x in row)
    
        # Create a mask to filter out rows with complex elements close to zero
        mask = np.array([not is_complex_close_to_zero(row) for row in array])
    
        # Return the real parts of the filtered array
        return np.real(array[mask])
    return remove_near_zero_complex(r[:D,:].transpose(), tol)
    

In [162]:
nash_solver(game2, 1e-6)

array([[0.33333333, 0.33333333, 0.33333333, 0.33333333, 0.33333333,
        0.33333333]])

In [320]:
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s) + 1))


In [325]:
def potential_support_pairs(game):
    num_strategies = game.num_strategies
    support_sets = [sorted(list(powerset(range(num))), key=lambda x: (len(x), x)) for num in num_strategies]

    combined_supports_list = list(product(*support_sets))
    sorted_combined_supports_list = sorted(combined_supports_list, key=lambda x: (sum(len(y) for y in x), max(len(y) for y in x) - min(len(y) for y in x), x))

    for sorted_combined_supports in sorted_combined_supports_list:
        yield sorted_combined_supports

In [327]:
for i in potential_support_pairs(game3):
    print(i)

((0,), (0,), (0,))
((0,), (0,), (1,))
((0,), (1,), (0,))
((0,), (1,), (1,))
((1,), (0,), (0,))
((1,), (0,), (1,))
((1,), (1,), (0,))
((1,), (1,), (1,))
((2,), (0,), (0,))
((2,), (0,), (1,))
((2,), (1,), (0,))
((2,), (1,), (1,))
((0,), (0,), (0, 1))
((0,), (0, 1), (0,))
((0,), (0, 1), (1,))
((0,), (1,), (0, 1))
((0, 1), (0,), (0,))
((0, 1), (0,), (1,))
((0, 1), (1,), (0,))
((0, 1), (1,), (1,))
((0, 2), (0,), (0,))
((0, 2), (0,), (1,))
((0, 2), (1,), (0,))
((0, 2), (1,), (1,))
((1,), (0,), (0, 1))
((1,), (0, 1), (0,))
((1,), (0, 1), (1,))
((1,), (1,), (0, 1))
((1, 2), (0,), (0,))
((1, 2), (0,), (1,))
((1, 2), (1,), (0,))
((1, 2), (1,), (1,))
((2,), (0,), (0, 1))
((2,), (0, 1), (0,))
((2,), (0, 1), (1,))
((2,), (1,), (0, 1))
((0,), (0, 1), (0, 1))
((0, 1), (0,), (0, 1))
((0, 1), (0, 1), (0,))
((0, 1), (0, 1), (1,))
((0, 1), (1,), (0, 1))
((0, 2), (0,), (0, 1))
((0, 2), (0, 1), (0,))
((0, 2), (0, 1), (1,))
((0, 2), (1,), (0, 1))
((1,), (0, 1), (0, 1))
((1, 2), (0,), (0, 1))
((1, 2), (0, 1)

In [314]:
def payoff_player(payoff, indices):
    #print("Payoffs:", payoff, "Indices:", indices)
    indices = list(indices)
    index = indices.pop(0)
    t_payoff = [payoff[i] for i in index]
    for i, pay in enumerate(t_payoff):
        if len(indices)>0:
            pay_1 = payoff_player(pay, indices)
            t_payoff[i] = pay_1
        else:
            t_payoff[i] = pay
    return t_payoff

def block_game(payoffs, indices):
    block = []
    for p, payoff in enumerate(payoffs):
        index =  [*indices]
        index.pop(p)
        indexes = tuple([indices[p], *index])
        block_n = payoff_player(payoff, indexes)
        block.append(block_n)
    return block

In [313]:
block_game(payoffs3,((1,2,), (0,1),(0,)))

[[[[0], [9]], [[0], [2]]], [[[0], [2]], [[8], [5]]], [[[0, 2], [1, 1]]]]

In [316]:
for block in potential_support_pairs(game4):
    print(block_game(payoffs4, block))

[[[[[1]]]], [[[[17]]]], [[[[33]]]], [[[[49]]]]]
[[[[[2]]]], [[[[18]]]], [[[[34]]]], [[[[57]]]]]
[[[[[3]]]], [[[[19]]]], [[[[41]]]], [[[[50]]]]]
[[[[[4]]]], [[[[20]]]], [[[[42]]]], [[[[58]]]]]
[[[[[5]]]], [[[[25]]]], [[[[35]]]], [[[[51]]]]]
[[[[[6]]]], [[[[26]]]], [[[[36]]]], [[[[59]]]]]
[[[[[7]]]], [[[[27]]]], [[[[43]]]], [[[[52]]]]]
[[[[[8]]]], [[[[28]]]], [[[[44]]]], [[[[60]]]]]
[[[[[9]]]], [[[[21]]]], [[[[37]]]], [[[[53]]]]]
[[[[[10]]]], [[[[22]]]], [[[[38]]]], [[[[61]]]]]
[[[[[11]]]], [[[[23]]]], [[[[45]]]], [[[[54]]]]]
[[[[[12]]]], [[[[24]]]], [[[[46]]]], [[[[62]]]]]
[[[[[13]]]], [[[[29]]]], [[[[39]]]], [[[[55]]]]]
[[[[[14]]]], [[[[30]]]], [[[[40]]]], [[[[63]]]]]
[[[[[15]]]], [[[[31]]]], [[[[47]]]], [[[[56]]]]]
[[[[[16]]]], [[[[32]]]], [[[[48]]]], [[[[64]]]]]
[[[[[1, 2]]]], [[[[17, 18]]]], [[[[33, 34]]]], [[[[49]]], [[[57]]]]]
[[[[[3, 4]]]], [[[[19, 20]]]], [[[[41, 42]]]], [[[[50]]], [[[58]]]]]
[[[[[1], [3]]]], [[[[17], [19]]]], [[[[33]]], [[[41]]]], [[[[49, 50]]]]]
[[[[[2], [4]]]